In [11]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
import joblib
import swifter
import re

In [2]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# Remove double quotes from song names
df['song'] = df['song'].str.replace("\"", "")
# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021


In [3]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max'})

# Create separate Billboard dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of Billboard dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

In [4]:
# SPOTIFY API
# Import dependencies for Spotipy package
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Import Client ID and Client Secret from config.py
from config import cid, secret

# Initialize objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# Searches spotify API by song and artist and returns Track ID
def search_spotify(song_title, artist):
    try:
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
        track_id = searchResults['tracks']['items'][0]['id']

        return track_id
    except:
        pass

In [6]:
# Returns list of different song title combinations with/without/inside/outside parentheses
def adjust_parens(song_title):
    # Initialize empty list for song string combos
    combinations = []

    # Remove parentheses from song title
    strip_parens = ' '.join(song_title.strip(')(').split(')'))
    strip_parens = ' '.join(strip_parens.strip(' (').split('(')).replace('  ',' ')

    # Add song title 
    combinations.append(strip_parens)

    # 
    try:
        inside_parens = re.findall(r'\(.*?\)', song_title)[0].strip('()').strip()
    except IndexError:
        pass
        
    try:
        left_parens = song_title.split(f"({inside_parens})")[0].strip()
        if (left_parens not in combinations) & (len(left_parens) > 0): 
            combinations.append(left_parens)
    except:
        left_parens = "NA"

    try:
        right_parens = song_title.split(f"({inside_parens})")[1].strip()
        if (right_parens not in combinations) & (len(right_parens) > 0): 
            combinations.append(right_parens)
    except:
        right_parens = "NA"

    return combinations

In [7]:
# Retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    while True:
        # Select first artist if multiple listed with "Featuring" keyword
        if 'Featuring' in artist:
            artist = artist.split(' Featuring ')[0]
        # Select first artist if multiple listed with "with", "With", or "," substrings
        elif ' with ' in artist:
            artist = artist.split(' with ')[0]
        elif ' With ' in artist:
            artist = artist.split(' With ')[0]
        elif "," in artist:
            artist = artist.split(',')[0]

        # Make initial API search, return ID string if found
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        # Search artist and song title (replacing words ending in "in" to "ing")
        song_title = re.sub(r"in\b", 'ing ', song_title)
        found_id = search_spotify(song_title, artist)
        if found_id:
            return found_id

        if '(' in song_title:
            for item in adjust_parens(song_title):
                found_id = search_spotify(item, artist)
                if found_id:
                    return found_id
                
        # Check for '/' character in song_title
        if '/' in song_title:
            # Try string on left side of '/'
            song_title = song_title.split('/')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id

            # Try string on right side of '/'
            try:
                song_title = song_title.split('/')[1]
                found_id = search_spotify(song_title, artist)
                if found_id:
                    return found_id
            except:
                pass

        # Check for '&' character in artist name
        if ' & ' in artist:
            artist = artist.split(' & ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'X' character in artist name
        if ' X ' in artist:
            artist = artist.split(' X ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
        # Check for 'x' character in artist name
        if ' x ' in artist:
            artist = artist.split(' x ')[0]
            found_id = search_spotify(song_title, artist)
            if found_id:
                return found_id
            
        # Print song title and artist for non-match
        if found_id:
            return found_id
        
        break
    

In [8]:
# Returns tuple of audio features from Spotify for specified track_id
def get_audio_features(id):
    try:
        # Reach out to spotify API
        search_results = sp.audio_features(id)[0]

        # Store returned audio features
        danceability = search_results['danceability']
        energy = search_results['energy']
        loudness = search_results['loudness']
        speechiness = search_results['speechiness']
        acousticness = search_results['acousticness']
        instrumentalness = search_results['instrumentalness']
        liveness = search_results['liveness']
        valence = search_results['valence']
        tempo = search_results['tempo']
        duration_ms = search_results['duration_ms']
    except:
        # Return null values if search unsuccessful
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Return list of features
    return [danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, \
                valence, tempo, duration_ms]

In [11]:
# Add new "track_id" column for 1960's dataset based on Spotify API calls and export as csv
billboard_1960s['track_id'] = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv', index=False)

In [64]:
# Add new "track_id" column for 1970's dataset based on Spotify API calls and export as csv
billboard_1970s['track_id'] = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv', index=False)

Pandas Apply:   0%|          | 0/6251 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 1980's dataset based on Spotify API calls and export as csv
billboard_1980s['track_id'] = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv', index=False)

Pandas Apply:   0%|          | 0/5126 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Add new "track_id" column for 1990's dataset based on Spotify API calls and export as csv
billboard_1990s['track_id'] = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv', index=False)

Pandas Apply:   0%|          | 0/4405 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2000's dataset based on Spotify API calls and export as csv
billboard_2000s['track_id'] = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv', index=False)

Pandas Apply:   0%|          | 0/4433 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Add new "track_id" column for 2010's dataset based on Spotify API calls and export as csv
billboard_2010s['track_id'] = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv', index=False)

Pandas Apply:   0%|          | 0/5582 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# Add new "track_id" column for 2020's dataset based on Spotify API calls and export as csv
billboard_2020s['track_id'] = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv', index=False)

Pandas Apply:   0%|          | 0/1479 [00:00<?, ?it/s]

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
# Read in each saved decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s.csv')

In [9]:
# Create dataframe of all billboard data
all_billboard_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])

# Export to csv
all_billboard_df.to_csv('./Resources/billboard_ALL.csv', index=False)

In [10]:
# Get audio features for each dataframe and export as separate csv
def add_features_to_df(df, file_name):
    # Drop track ID
    df.dropna(subset=['track_id'], inplace=True)
    
    # Get audio features from the track ID of each row in dataframe
    df['danceability'], df['energy'], df['key'], df['loudness'], df['mode'], df['speechiness'], df['acousticness'], \
    df['instrumentalness'], df['liveness'], df['valence'], df['tempo'], df['duration_ms'], df['time_signature'] = \
                                          zip(*df['track_id'].map(get_audio_features))

    # Export to csv
    df.to_csv(f'./Resources/{file_name}_features.csv', index=False)

In [108]:
# Add feature columns to each decade dataframe and export as new csv file
add_features_to_df(df_2020s, "billboard_2020s")
add_features_to_df(df_2010s, "billboard_2010s")
add_features_to_df(df_2000s, "billboard_2000s")
add_features_to_df(df_1990s, "billboard_1990s")
add_features_to_df(df_1980s, "billboard_1980s")
add_features_to_df(df_1970s, "billboard_1970s")
add_features_to_df(df_1960s, "billboard_1960s")

In [6]:
# Read in each decade csv file for Billboard data
df_1960s = pd.read_csv('./Resources/billboard_1960s_features.csv')
df_1970s = pd.read_csv('./Resources/billboard_1970s_features.csv')
df_1980s = pd.read_csv('./Resources/billboard_1980s_features.csv')
df_1990s = pd.read_csv('./Resources/billboard_1990s_features.csv')
df_2000s = pd.read_csv('./Resources/billboard_2000s_features.csv')
df_2010s = pd.read_csv('./Resources/billboard_2010s_features.csv')
df_2020s = pd.read_csv('./Resources/billboard_2020s_features.csv')

In [7]:
# Combine all complete billboard dfs into one and export to csv file
all_billboard_features_df = pd.concat([df_1960s, df_1970s, df_1980s, df_1990s, df_2000s, df_2010s, df_2020s])
# Drop column (added earlier by mistake)
all_billboard_features_df.drop("weeks-at-no1", axis=1, inplace=True)
# Drop rows with missing values
all_billboard_features_df.dropna(inplace=True)
# Export to csv
all_billboard_features_df.to_csv('./Resources/billboard_ALL_features.csv', index=False)

In [74]:
# Read in 1.2 Million Spotify Songs dataset and cleaned billboard csv
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')
all_billboard_features_df = pd.read_csv('./Resources/billboard_ALL_features.csv')

In [63]:
# Rename columns to match billboard data
spotify_1m = spotify_1m.rename(columns={"id":"track_id", "name":"song"})
# Set default values for non-charting songs to 0
spotify_1m[['peak-rank', 'weeks-on-board', 'billboard']] = 0
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")
# Filter to selected columns
spotify_1m_filtered = spotify_1m[['song', 'artist', 'year', 'peak-rank', 'weeks-on-board', 'track_id',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'billboard']]

# Preview dataframe
spotify_1m_filtered.head()

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard
0,Testify,Rage Against The Machine,1999,0,0,7lmeHLHBe4nmXzuXc0HDjk,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,0
1,Guerrilla Radio,Rage Against The Machine,1999,0,0,1wsRitfRRtWyEapl0q22o8,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,0
2,Calm Like a Bomb,Rage Against The Machine,1999,0,0,1hR0fIFK2qRG3f3RF70pb7,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,0
3,Mic Check,Rage Against The Machine,1999,0,0,2lbASgTSoDO7MTuLAXlTW0,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,0
4,Sleep Now In the Fire,Rage Against The Machine,1999,0,0,1MQTmpYOZ6fcMQc56Hdo7T,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,0


In [64]:
# Combine charting and non-charting songs
all_songs_df = pd.concat([all_billboard_features_df, spotify_1m_filtered])
# Remove any duplicate track IDs
all_songs_df = all_songs_df.drop_duplicates(subset=['track_id', 'year'])
all_songs_df.head()

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,1.0,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,1.0,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,1.0,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,1.0,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,1.0,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1


In [65]:
def get_decade(row):
    if (row['year']<1970) & (row['year']>=1960):
        return "1960s"
    elif (row['year']<1980) & (row['year']>=1970):
        return "1970s"
    elif (row['year']<1990) & (row['year']>=1980):
        return "1980s"
    elif (row['year']<2000) & (row['year']>=1990):
        return "1990s"
    elif (row['year']<2010) & (row['year']>=2000):
        return "2000s"
    elif (row['year']<2020) & (row['year']>=2010):
        return "2010s"
    elif (row['year']<2020) & (row['year']>=2010):
        return "2020s"

In [66]:
# Create decade column for each song
all_songs_df['decade'] = all_songs_df.swifter.apply(lambda row:get_decade(row),axis=1)

# Preview dataframe
all_songs_df.head()

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/swifter/swifter.py:88: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, "


Pandas Apply:   0%|          | 0/1233416 [00:00<?, ?it/s]

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard,decade
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,...,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1,1960s
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,...,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1,1960s
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,...,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1,1960s
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,...,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1,1960s
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,...,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1,1960s


In [67]:
# Export complete dataset to csv
all_songs_df.to_csv("./Resources/all_songs_final.csv", index=False)

In [77]:
import json

from sklearn.preprocessing import MinMaxScaler, StandardScaler
features_by_year_df = all_billboard_features_df.groupby('year', as_index=False).median().drop(columns=['key', 'mode', 'time_signature', 'billboard', 'peak-rank'])
features_by_year_df


features_by_year_df

features_by_year_df = features_by_year_df.rename(columns={'weeks-on-board': 'weeks_on_board'})
features_by_year_json = features_by_year_df.to_dict('list')
features_by_year_json

with open('./static/js/featuresByYear.js', 'w') as f:
    f.write('let featuresByYear=')
    json.dump(features_by_year_json, f)

In [78]:
# Add decade column to billboard features df
all_billboard_features_df['decade'] = all_billboard_features_df.swifter.apply(lambda row:get_decade(row),axis=1)
# Display df
all_billboard_features_df.head()

Pandas Apply:   0%|          | 0/31874 [00:00<?, ?it/s]

,song,artist,year,peak-rank,weeks-on-board,track_id,danceability,energy,key,loudness,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,billboard,decade
0,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,4wXNtDvLZl55vlQashWQrI,0.686,0.562,5.0,-8.538,...,0.0504,0.589,0.000053,0.124,0.793,116.782,156886.0,4.0,1,1960s
1,(All Of A Sudden) My Heart Sings,Mel Carter,1965,38,7,6x5BiQwNlbtisITsEHa8Eu,0.419,0.546,7.0,-7.819,...,0.0363,0.479,0.000005,0.535,0.493,118.106,164067.0,3.0,1,1960s
2,(Baby) Hully Gully,The Olympics,1960,72,7,2CkbNJooPxUsHG4gFXKhxa,0.699,0.661,2.0,-6.090,...,0.0382,0.516,0.000000,0.198,0.869,115.772,218642.0,4.0,1,1960s
3,(Come round Here) Im The One You Need,The Miracles,1966,17,9,5YvokzRkNONpj5c0TIqTjQ,0.545,0.607,0.0,-9.525,...,0.0347,0.242,0.000000,0.779,0.767,113.880,150067.0,4.0,1,1960s
4,(Dance The) Mess Around,Chubby Checker,1961,24,7,58EirkLFB6O6wwRN9JowHB,0.511,0.694,10.0,-5.509,...,0.0479,0.381,0.000000,0.195,0.826,146.559,132933.0,4.0,1,1960s


In [176]:
scaled_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                   'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
grouped_decade_df = all_billboard_features_df.drop(columns=['year', 'billboard'])

# Initialize scaler
feature_scaler = MinMaxScaler()
# Fit scaler to desired features
feature_scaler.fit(all_billboard_features_df[scaled_features])
# Transform features
grouped_decade_df[scaled_features] = feature_scaler.transform(all_billboard_features_df[scaled_features])

# Group dataframe by decade to get median feature values
grouped_decade_df = grouped_decade_df.groupby('decade').median()
grouped_decade_dict = grouped_decade_df.to_dict('index')
grouped_decade_df.head()
with open('./static/js/groupedDecades.js', 'w') as f:
    f.write('let groupedDecades=')
    json.dump(grouped_decade_dict, f)


In [177]:
import pickle
# create an iterator object with write permission - model.pkl
with open('./Resources/audio_feature_scaler.pickle', 'wb') as files:
    pickle.dump(feature_scaler, files)

In [16]:
def get_audio_features(id):
    # Create list of desired audio features
    features_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                   'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    # Make call to spotify API by ID to return dictionary of audio features
    search_results = sp.audio_features(id)[0]
    # Check if search returned anything
    if search_results:
        # Create list of values for features in features_list
        features = [v for k,v in search_results.items() if k in features_list]
        # Create dictionary for returned audio features
        features_dict = {features_list[i]:features[i] for i in range(len(features_list))}
        # Create dataframe from dictionary to apply scaler
        feat_df = pd.DataFrame(features_dict, index=[0])
        # Load in saved scaler
        audio_feature_scaler = joblib.load("./Resources/audio_feature_scaler.pickle")
        # Transform audio features in dataframe into array
        scaled_features = audio_feature_scaler.transform(feat_df)[0]

        # Create new dictionary with scaled features
        scaled_features_dict = {features_list[i]:scaled_features[i] for i in range(len(features_list))}
        # Return dictionary with scaled features
        return scaled_features_dict
    # Return none
    return

In [17]:
get_audio_features("3AuzZHPlohKLpildLyORSM")

{'danceability': 0.5850202429149797,
 'energy': 0.42770475227502525,
 'loudness': 0.7418574011091706,
 'speechiness': 0.03207150368033649,
 'acousticness': 0.33165679356246197,
 'instrumentalness': 0.0,
 'liveness': 0.07357723577235774,
 'valence': 0.2906155398587285,
 'tempo': 0.32775124580409865,
 'duration_ms': 0.21212257738151696}